# pdf转txt

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 23 16:17:30 2019

@author: Administrator
"""
#传入文件路径名，将所有pdf转为同名的txt文件保存

import sys
import importlib
importlib.reload(sys)

from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal, LAParams
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed
import os

class PdfToTxt(object):
    
    #从已知文件路径中读取pdf文件，并将其转化为同名txt文件

    def readPDF(path, toPath):
        # 以二进制形式打开pdf文件
   
        with open(path, "rb") as f:
            # 创建一个pdf文档分析器
            parser = PDFParser(f)
            # 创建pdf文档
            pdfFile = PDFDocument()
            # 链接分析器与文档对象
            parser.set_document(pdfFile)
            pdfFile.set_parser(parser)
            # 提供初始化密码
            pdfFile.initialize()
            # 检测文档是否提供txt转换
        if not pdfFile.is_extractable:
            raise PDFTextExtractionNotAllowed
        else:
            # 解析数据
            # 数据管理
            manager = PDFResourceManager()
            # 创建一个PDF设备对象
            laparams = LAParams()
            device = PDFPageAggregator(manager, laparams=laparams)
            # 解释器对象
            interpreter = PDFPageInterpreter(manager, device)
    
            # 开始循环处理，每次处理一页
            for page in pdfFile.get_pages():
                interpreter.process_page(page)
                layout = device.get_result()
                for x in layout:
                    if(isinstance(x, LTTextBoxHorizontal)):
                        with open(toPath, 'a',encoding='utf-8') as f:
                            str = x.get_text()
                            # print(str)
                            f.write(str+"\n")

    #遍历文件夹及其子文件夹
    def list_all_files(rootdir):
        _files = []
        list = os.listdir(rootdir) #列出文件夹下所有的目录与文件
        for i in range(0,len(list)):
            path = os.path.join(rootdir,list[i])
            if os.path.isdir(path):
                _files.extend(PdfToTxt.list_all_files(path))
            if os.path.isfile(path):
                _files.append(path)
        return _files
    #将传入的文件夹及其子文件夹下的pdf文件转为txt文件，并存到固定路径下
    def getpdf_list(pdf_path):
        txt_path =  'C:/EdmsData/txt1/'
        #清空txt文件中文件
        shanchu = os.listdir(txt_path)
        #print(shanchu)
        for i in range(len(shanchu)):
            os.remove(txt_path+shanchu[i])
        #--------------------------------
        all_list = PdfToTxt.list_all_files(pdf_path)
        path = ''
         
        txt_list=[]
        for onelist in all_list:
            if onelist.find('pdf') != -1:
                txt_list.append(onelist)
                
        print(txt_list)
        for i in range(len(txt_list)):
            path = txt_list[i]
            
            index = txt_list[i].rfind('\\')
            one_list = txt_list[i][index:-4]
            toPath = txt_path+one_list+'.txt'
            
            print(toPath)
            try:
                PdfToTxt.readPDF(path,toPath)
                
            except:
                print("error")
                continue
            
            
if __name__ == '__main__':
    pdf_path = 'C:/TMdata/PDF3'
    #将所有pdf转为同名的txt文件
    PdfToTxt.getpdf_list(pdf_path)
    

['C:/TMdata/PDF3\\2017年广州市体育产业增加值425_76亿元_黄心豪.pdf', 'C:/TMdata/PDF3\\_一街一站_让深圳大气治理走向精细化_陈昊.pdf']
C:/EdmsData/txt1/\2017年广州市体育产业增加值425_76亿元_黄心豪.txt
C:/EdmsData/txt1/\_一街一站_让深圳大气治理走向精细化_陈昊.txt


In [9]:
# -*- coding: utf-8 -*-
"""
Created on Sat May  4 20:06:33 2019

@author: Administrator
"""
from sklearn.externals import joblib
from MyDataFrame import MyDataFrame
class GetCatagory(object):
    def getAllTextCatagory(self):
        nbf =joblib.load('LR_model')  # 加载Logistic回归模型
        mdf = MyDataFrame()
        df = mdf.new_DataFrame()
        df2 = mdf.m_cut(df)
        rawfile=[]#空格分隔开的文本
        for i in range(len(df2)):
            rawfile.append(' '.join(df2['fenci'][i]))

        countvec=joblib.load('countvec_LR')  #加载Logistic回归模型拟合时的原始词频特征矩阵
        x_test_wordmtx = countvec.transform(rawfile) 
        transformer = joblib.load('transformer_LR')
        x_test = transformer.transform(x_test_wordmtx)
        catagory_list = nbf.predict(x_test) #对测试集进行类别预测，返回一个list
        #print(catagory_list)
        #print(type(catagory_list))
        return catagory_list
    
    """
    def getOneTextCatagory(self,filename):
        catagory_list = self.getAllTextCatagory()
    """ 
        
    
if __name__ == '__main__':
    gc = GetCatagory()
    catagory_list = gc.getAllTextCatagory()  # list
    print(catagory_list)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
DEBUG:jieba:Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.155 seconds.
DEBUG:jieba:Loading model cost 1.155 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


['Sports' 'Economy' 'Politics' 'Computer' 'Economy' 'Economy' 'Sports'
 'Art' 'Computer' 'Agriculture' 'Enviornment' 'Art' 'Computer' 'Computer'
 'Sports' 'Sports' 'Computer' 'Computer' 'Economy' 'Agriculture'
 'Agriculture' 'Economy' 'Agriculture' 'Agriculture' 'Economy' 'Sports'
 'Economy' 'Computer' 'Computer' 'Computer' 'Computer' 'Politics' 'Sports'
 'Computer' 'Agriculture' 'Agriculture' 'Agriculture' 'Computer' 'Sports'
 'Economy' 'Sports' 'Computer' 'Agriculture' 'Art' 'Computer' 'Computer'
 'Politics' 'Computer' 'Computer' 'Economy' 'Economy' 'Computer' 'Computer'
 'Art' 'Politics']
